In [1]:
import pandas as pd
import numpy as np

In [2]:
complaints = pd.read_csv('data.csv')

In [3]:
#Q1: calculating the unique number of complaints which does not have NaN values
NaN_rows = complaints[complaints.isnull().any(1)]
new_data = complaints[~complaints.index.isin(NaN_rows.index)]
new_data.nunique()

DateStamp                                          1
UniqueComplaintId                              68467
Close Year                                        12
Received Year                                     18
Borough of Occurrence                              6
Is Full Investigation                              2
Complaint Has Video Evidence                       2
Complaint Filed Mode                               7
Complaint Filed Place                             15
Complaint Contains Stop & Frisk Allegations        2
Incident Location                                 15
Incident Year                                     17
Encounter Outcome                                  4
Reason For Initial Contact                        49
Allegation FADO Type                               4
Allegation Description                            56
dtype: int64

In [4]:
#Q2
unique_borough = new_data.groupby(['Borough of Occurrence', 'UniqueComplaintId']).count()
a = unique_borough.reset_index().groupby('Borough of Occurrence')['UniqueComplaintId'].nunique()
print(a)

max(a)/len(unique_borough)

Borough of Occurrence
Bronx            15603
Brooklyn         23369
Manhattan        16067
Outside NYC         57
Queens           10608
Staten Island     2763
Name: UniqueComplaintId, dtype: int64


0.3413177151036266

In [5]:
#Q3 (Review again)
borough = ['Brooklyn', 'Bronx', 'Manhattan', 'Queens', 'Staten Island']
population = ['2648771', '1471160', '1664727', '2358582', '479458']
population = pd.DataFrame({'Borough of Occurrence': borough, 'Population': population})

incidents_2016 = new_data[new_data['Incident Year'] == 2016].groupby(['Borough of Occurrence', 'UniqueComplaintId']).count()
incidents_each_borough = incidents_2016.reset_index().groupby('Borough of Occurrence')['UniqueComplaintId'].nunique()
incidents_each_borough = pd.DataFrame(incidents_each_borough)

merged = pd.merge(incidents_each_borough, population, on ='Borough of Occurrence')
merged['Incidents per Capita'] = merged['UniqueComplaintId']/merged['Population'].astype(str).astype(int)
max_per_capita = merged[merged['Incidents per Capita'] == max(merged['Incidents per Capita'])]

pd.set_option('display.precision',10)

(max_per_capita['UniqueComplaintId']/max_per_capita['Population'].astype(str).astype(int))*100000


0    52.4755974877
dtype: float64

In [6]:
#Q4

ques_4 = new_data.copy()

ques_4['duration'] = ques_4['Close Year'] - ques_4['Received Year']
ques_4 = ques_4.groupby(['UniqueComplaintId', 'duration']).count()

np.array(ques_4.index.get_level_values('duration')).mean()

0.4743745161902814

In [7]:
#Q5

ques_5 = new_data.copy()
ques_5 = ques_5.groupby(['Incident Year', 'Complaint Contains Stop & Frisk Allegations']).count()
ques_5 = ques_5[ques_5.index.get_level_values('Complaint Contains Stop & Frisk Allegations') == True]
ques_5 = ques_5.reset_index(level=[0,1])
ques_5 = ques_5[(ques_5['Incident Year'] >= 2007) & (ques_5['Incident Year']<= 2016)]
final = ques_5.loc[:,['Incident Year','DateStamp']]
final.columns = ['Incident Year', 'Count']

import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
model = LinearRegression()

x_train = final['Incident Year'].values.reshape(10,1)
y_train = final['Count']

x_test = [[2017],[2018],[2019]]

model.fit(x_train, y_train)
y_pred = model.predict(x_test)

y_pred

array([2181.4       , 1368.90909091,  556.41818182])

In [8]:
ques6=new_data.groupby(['UniqueComplaintId','Complaint Has Video Evidence','Is Full Investigation'] ).count()
ques6=ques6.reset_index(level=[1,2])

from scipy import stats

crosstab = pd.crosstab(ques6['Is Full Investigation'], ques6['Complaint Has Video Evidence'])
crosstab

Complaint Has Video Evidence,False,True
Is Full Investigation,,
False,44529,584
True,21889,1465


In [9]:
stats.chi2_contingency(crosstab)

(1312.0319808202814,
 2.744813207558423e-287,
 1,
 array([[43762.91109586,  1350.08890414],
        [22655.08890414,   698.91109586]]))

In [10]:
new_data['Allegation FADO Type'].unique()

array(['Discourtesy', 'Force', 'Abuse of Authority', 'Offensive Language'],
      dtype=object)

In [11]:
ques7 = new_data.copy()
ques7['F'] = ''
ques7['A'] = ''
ques7['D'] = ''
ques7['O'] = ''

conditions = [ques7['Allegation FADO Type'] == 'Discourtesy',
              ques7['Allegation FADO Type'] == 'Force',
              ques7['Allegation FADO Type'] == 'Abuse of Authority',
              ques7['Allegation FADO Type'] == 'Offensive Language']
choices_D = [True,False,False,False]
choices_F = [False,True,False,False]
choices_A = [False,False,True,False]
choices_O = [False,False,False,True]

ques7['F'] = np.select(conditions, choices_F)
ques7['D'] = np.select(conditions, choices_D)
ques7['A'] = np.select(conditions, choices_A)
ques7['O'] = np.select(conditions, choices_O)

In [14]:
farshad = ques7.groupby(['UniqueComplaintId']).count()
farshad = farshad.reset_index(level=[0])

columns1 = ['UniqueComplaintId','F','A','D','O']
ali = ques7.loc[:,columns1]
ali2 = ali.groupby('UniqueComplaintId').sum()

ali2['Total Sum'] = ali2.sum(axis = 1)
ali2

import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split

model_F = LinearRegression()
model_A = LinearRegression()
model_D = LinearRegression()
model_O = LinearRegression()

X_train_F = ali2['F'].values.reshape(len(ali2),1)
Y_train_F = ali2['Total Sum']

X_train_A = ali2['A'].values.reshape(len(ali2),1)
Y_train_A = ali2['Total Sum']

X_train_D = ali2['D'].values.reshape(len(ali2),1)
Y_train_D = ali2['Total Sum']

X_train_O = ali2['O'].values.reshape(len(ali2),1)
Y_train_O = ali2['Total Sum']



model_F.fit(X_train_F, Y_train_F)
model_A.fit(X_train_A, Y_train_A)
model_D.fit(X_train_D, Y_train_D)
model_O.fit(X_train_O, Y_train_O)

print(model_F.coef_)
print(model_A.coef_)
print(model_D.coef_)
print(model_O.coef_)

np.set_printoptions(precision=10)

[1.0078341356364786]
[0.9893247344885125]
[1.4522933359966437]
[1.4701056259035243]


In [15]:
ques8 = pd.DataFrame(complaints[complaints['Incident Year'] == 2016].groupby(['Borough of Occurrence']).size()).reset_index()

In [16]:
ques8.columns = ['Borough of Occurrence', 'Number of Complaints']

In [17]:
ques8

,Borough of Occurrence,Number of Complaints
0,Bronx,2603
1,Brooklyn,3231
2,Manhattan,2316
3,Outside NYC,15
4,Queens,1646
5,Staten Island,493


In [18]:
population

,Borough of Occurrence,Population
0,Brooklyn,2648771
1,Bronx,1471160
2,Manhattan,1664727
3,Queens,2358582
4,Staten Island,479458


In [19]:
last = pd.merge(population, ques8)
last['Complaints per Capita'] = last['Number of Complaints']/last['Population'].astype(str).astype(int)
last['Number of Officers'] = round((last['Complaints per Capita']/last['Complaints per Capita'].sum())*36000)

last['Number of Precincts'] = [23,12,22,16,4]
last['Number of Officers per Precinct'] = round(last['Number of Officers']/last['Number of Precincts'])
last['Number of Officers per Precinct'].max()/last['Number of Officers per Precinct'].min()

5.898832684824903